# Narzędzie do importu do Excel punktów z Mapy Google na Bieg Nocny.
Żeby uruchomić program musisz kliknąć symbol startu obok komórek z programem. Wykonaj je **PO KOLEI**.

In [1]:
#@title Inicjalizacja
!pip install fastkml -q
from google.colab import files
import io
from fastkml import kml
import pandas as pd
import numpy as np

In [2]:
#@title Wgraj plik KML
#@markdown Żeby pobrać swoją Mapę Google jako .kml:
#@markdown - wybierz przycisk z symbolem trzech kropek -> pobierz KML
#@markdown - **zaznacz opcje Export do pliku KML zamiast KMZ**
#@markdown - uruchom komórkę i wgraj plik

#@markdown Wybierz id twojej mapy (zwykle 0)
uploaded = files.upload()

if not uploaded:
    raise ValueError("Please upload a file before continuing.")

filename = list(uploaded.keys())[0]

if not filename.lower().endswith('.kml'):
    raise ValueError("The uploaded file is not a KML file. Please upload a .kml file.")

try:
    kml_obj = kml.KML.parse(filename)
except Exception as e:
    raise ValueError("Could not read KML. Please check your file.") from e

map_id = 0 #@param {"type":"integer"}
map_to_import = kml_obj.features[0].features[0].features
print(f"Wczytano mapę {map_id} z pliku {filename} zawierającą {len(map_to_import)} punktów.")

Saving Trasa WI 2025.kml to Trasa WI 2025 (1).kml
Wczytano mapę 0 z pliku Trasa WI 2025 (1).kml zawierającą 120 punktów.


In [3]:
#@title Wczytaj plik Excel do wypełnienia
#@markdown Ustaw nazwę arkusza do którego chcesz importować.

#@markdown Następnie uruchom komórkę i wgraj plik .xlsx do wypełnienia.

route_sheet_name = "Trasa R" #@param {"type":"string"}

uploaded = files.upload()

if not uploaded:
    raise ValueError("Please upload a file before continuing.")

excel_filename = list(uploaded.keys())[0]

if not excel_filename.lower().endswith('.xlsx'):
    raise ValueError("The uploaded file is not a XLSX file. Please upload a .xlsx file.")

try:
    output_df = pd.read_excel(excel_filename, sheet_name=route_sheet_name, engine='openpyxl')
except Exception as e:
    raise ValueError("Could not read XLSX. Please check your file.") from e


output_df.head()

Saving Punkty puste.xlsx to Punkty puste (1).xlsx


,NR Punktu,Kod,Nazwa Punktu,Współrzędna N,Współrzędna E,ID Pytania,Odpowiedź


In [4]:
#@title Wypełnianie tabeli
#@markdown Po uruchomieniu komórka wypełni tabelę punktami z mapy.

#@markdown Wybierz pierwszą literę kodu punktu - powninna być to inna litra dla każdej z tras w danej edycji.

#@markdown Ustaw czy chcesz żeby program dodał id pytań (po kolei) oraz czy ma dodać losowe odpowiedzi (ABCD)

point_code_first_letter = 'A' # @param {"type":"string"}
point_codes = [f"{point_code_first_letter}{chr(65 + i)}{chr(65 + j)}" for i in range(26) for j in range(26)]
np.random.shuffle(point_codes)
add_question_ids = True #@param {"type":"boolean"}
add_random_answers = True #@param {"type":"boolean"}

for i, placemark in enumerate(map_to_import):
    row = dict()
    row['NR Punktu'] = i + 1
    row['Kod'] = point_codes[i]
    row['Nazwa Punktu'] = placemark.name
    row['Współrzędna N'] = placemark.geometry.coords[0][1]
    row['Współrzędna E'] = placemark.geometry.coords[0][0]
    if add_question_ids:
        row['ID Pytania'] = f"P{i+1}"
    if add_random_answers:
        row['Odpowiedź'] = np.random.choice(['A', 'B', 'C', 'D'])

    output_df.loc[i] = row
output_df.head()


,NR Punktu,Kod,Nazwa Punktu,Współrzędna N,Współrzędna E,ID Pytania,Odpowiedź
0,1,AYQ,Hufiec ZHP Poznań-Rejon,52.394037,16.903264,P1,B
1,2,AOS,Emilii Sczanieckiej,52.395139,16.895029,P2,D
2,3,ATD,Szymborska,52.392371,16.888367,P3,D
3,4,ARL,Stadion Grunwaldu,52.396670,16.883460,P4,D
4,5,AUB,Kościół św. Jana Kantego,52.398094,16.875550,P5,C


In [5]:
#@title Zapisz i pobierz
with pd.ExcelWriter(excel_filename, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    output_df.to_excel(writer, sheet_name=route_sheet_name, index=False)

files.download(excel_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>